# Olmo 3 7B SFT Workflow

Complete workflow for training OLMo-3-7B with supervised fine-tuning using OLMo-core.

## Step 1: Clone OLMo-core repository


In [ ]:
%%bash
set -euo pipefail

REPO_DIR="/work/dlclarge2/ferreira-oellm/OLMo-core"
if [ ! -d "$REPO_DIR" ]; then
  git clone https://github.com/allenai/OLMo-core "$REPO_DIR"
fi
cd "$REPO_DIR"
git fetch origin
git checkout jacobm-final-merge-sft
git pull origin jacobm-final-merge-sft


## Step 2: Prepare SFT dataset

Convert your SFT data to OLMo-core's tokenized format.


In [ ]:
!./oellm/sft/prepare_dolci_sft_data.sh /work/dlclarge2/ferreira-oellm/open-instruct/data/dolci_think_sft_tokenized


## Step 3: Download base model checkpoint


In [ ]:
!python oellm/sft/download_olmo3_7b.py --output-dir models/Olmo-3-1025-7B


## Step 4: Convert HuggingFace checkpoint to OLMo-core format

The downloaded HuggingFace checkpoint needs to be converted to OLMo-core's native format (.distcp files) before it can be used for training.


In [ ]:
%%bash
set -euo pipefail

OLMOCORE_PATH="/work/dlclarge2/ferreira-oellm/OLMo-core"
export PYTHONPATH="${OLMOCORE_PATH}/src:${PYTHONPATH:-}"

# Convert HuggingFace checkpoint to OLMo-core format
python "${OLMOCORE_PATH}/src/examples/huggingface/convert_checkpoint_from_hf.py" \
  -i allenai/Olmo-3-1025-7B \
  -m olmo3_7b \
  -o models/Olmo-3-1025-7B-olmocore

# Copy metadata to root for checkpoint recognition
cp models/Olmo-3-1025-7B-olmocore/model_and_optim/.metadata models/Olmo-3-1025-7B-olmocore/.metadata

echo "Conversion complete! Checkpoint saved to models/Olmo-3-1025-7B-olmocore"


## Step 5: Launch SFT Training

Now that the dataset is prepared and the checkpoint is converted, you can launch the training:

**Notes:**
- The script uses the converted OLMo-core checkpoint at `models/Olmo-3-1025-7B-olmocore`
- Logs will be saved to `slurm_logs/olmo3-7b-think-sft/`
- Training checkpoints will be saved to `checkpoints/ferreira/olmo3-7b-sft/dolci-think-sft/`
- The script is configured for 2 GPUs by default (for testing). For full training, use 8 GPUs on the `alldlc2_gpu-h200` partition



In [ ]:
!sbatch oellm/sft/train_think_sft_dolci_7b_slurm.sh
